> Implement Linear Regression by PyTorch as much as possible.

In [1]:
import os
import pickle
import torch

from torch.utils import data
from torch import nn

from typing import Dict, Tuple, List, Iterator, Any

# Load Data

In [2]:
linreg_data_path: str = os.path.join('..', '..', 'data', 'linreg_data', 'linreg_data.pkl')

with open(linreg_data_path, 'rb') as f:
    pkl_data: Dict[str, torch.Tensor] = pickle.load(f)

print(pkl_data.keys())
print({key:value_type for key, value_type in zip(pkl_data.keys(), map(type, pkl_data.values()))})

dict_keys(['true_w', 'true_b', 'features', 'labels'])
{'true_w': <class 'torch.Tensor'>, 'true_b': <class 'torch.Tensor'>, 'features': <class 'torch.Tensor'>, 'labels': <class 'torch.Tensor'>}


In [3]:
true_w, true_b = pkl_data['true_w'], pkl_data['true_b']
features, labels = pkl_data['features'], pkl_data['labels']

In [4]:
def build_dataloader(tensor_dataset: Tuple[torch.Tensor], batch_size: int, is_train: bool = True) -> Any:
    """Build PyTorch DataLoader."""
    dataset: data.TensorDataset = data.TensorDataset(*tensor_dataset)
    dataloader: data.DataLoader = data.DataLoader(dataset, batch_size=batch_size, shuffle=is_train)
    return dataloader

# Build Model Architecture

In [5]:
# Build linear regression model
linreg: nn.Sequential = nn.Sequential(nn.Linear(2, 1))

In [6]:
# Initialize model parameters
linreg[0].weight.data.normal_(mean=0, std=1e-2)
linreg[0].bias.data.fill_(0)

tensor([0.])

# Train Model

In [7]:
# Use MSE as the loss function
mse: Any = nn.MSELoss(reduction='mean')

In [8]:
# Use sgd as the optimizer
sgd: Any = torch.optim.SGD

In [9]:
# Setup hyper-parameters
batch_size: int = 10
lr: float = 3e-2
num_epochs: int = 5
net: Any = linreg
loss: Any = mse
trainer: Any = sgd(net.parameters(), lr=lr)

In [ ]:
# Build model training procedure
for epoch in range(num_epochs):
    for X, y in build_dataloader((features, labels), batch_size):
        l: float = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    with torch.no_grad():
        train_loss: float = loss(net(features), labels)
        print(f'epoch {epoch + 1}, loss {float(train_loss):f}')

epoch 1, loss 0.000235
epoch 2, loss 0.000097
epoch 3, loss 0.000097
epoch 4, loss 0.000097
epoch 5, loss 0.000099


# Verify Training Result

In [11]:
w: torch.Tensor = net[0].weight.data
b: torch.Tensor = net[0].bias.data

In [12]:
print(f"w estimated: {w.reshape(true_w.shape)}, true w: {true_w}")
print(f"b estimated: {b}, true b: {true_b}")

print(f"w and true_w difference: {torch.abs(true_w - w.reshape(true_w.shape))}")
print(f"b and true_b difference: {torch.abs(true_b - b)}")

w estimated: tensor([ 2.0000, -3.4009]), true w: tensor([ 2.0000, -3.4000])
b estimated: tensor([4.1996]), true b: tensor([4.2000])
w and true_w difference: tensor([3.4094e-05, 9.4533e-04])
b and true_b difference: tensor([0.0004])
